# Extraction Demo 

With this notebook, light-field decomposition to Sub-Aperture Images (SAIs) and refocused image slices is presented given an aligned light-field in form of a pickle file rendered as shown in [02_align_demo.ipynb](./02_align_demo.ipynb) using **[PlenoptiCam](https://github.com/hahnec/plenopticam)**.

## Package and import prerequisites

In [ ]:
import sys, os
print('Python v'+sys.version+'\n')

try:
    import plenopticam as pcam
except ImportError:
    !pip install plenopticam>=0.5.3
    import plenopticam as pcam
print('PlenoptiCam v'+pcam.__version__+'\n')

try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib --upgrade
    import matplotlib.pyplot as plt
    
%matplotlib inline

## Image data acquisition

Available plenoptic photographs can be downloaded to the current folder ('./data'), using the featured `DataDownloader` class. This is also used for extracting archived files.

In [ ]:
loader = pcam.misc.DataDownloader()
loader.download_data(loader.opex_url, fp='./data')
fnames = loader.opex_fnames_wht + loader.opex_fnames_lfp
loader.extract_archive(archive_fn='./data/5201452', fname_list=fnames)

## Configuration of *PlenoptiCam*

Before running the process, configurations such as file paths need to be set, using the `PlenopticamConfig` class as follows

In [ ]:
# instantiate config object and set image file paths and options
cfg = pcam.cfg.PlenopticamConfig()
cfg.default_values()
cfg.params[cfg.lfp_path] = './data/f197Inf9pxFinalShift12.7cm.bmp'
cfg.params[cfg.cal_meta] = './data/f197Inf9pxFinalShift12.7cmf22.json'
cfg.params[cfg.ptc_leng] = 11

# load calibration data
cfg.load_cal_data()

# instantiate status object to display processing progress
sta = pcam.misc.PlenopticamStatus()

## Load aligned light-field

In [ ]:
try:
    import pickle
    # load previously computed light field alignment
    fp = os.path.join(cfg.exp_path, 'lfp_img_align.pkl')
    lfp_img_align = pickle.load(open(fp, 'rb'))
except EOFError:
    os.remove(fp)

## Sub-Aperture Image (SAI) extraction

Rendering perspective views, known as SAIs, from an aligned light-field image `lfp_img_align` is accomplished by a series of classes in the `lfp_extractor` module. The resulting `vp_img_arr` is displayed in various ways further below.

In [ ]:
# micro image crop
lfp_obj = pcam.lfp_extractor.LfpCropper(lfp_img_align=lfp_img_align, cfg=cfg, sta=sta)
lfp_obj.main()
lfp_img_align = lfp_obj.lfp_img_align

# rearrange light-field to sub-aperture images
if cfg.params[cfg.opt_view]:
    lfp_obj = pcam.lfp_extractor.LfpRearranger(lfp_img_align, cfg=cfg, sta=sta)
    lfp_obj.main()
    vp_img_linear = lfp_obj.vp_img_arr

# remove outliers if option is set
if cfg.params[cfg.opt_lier]:
    obj = pcam.lfp_extractor.LfpOutliers(vp_img_arr=vp_img_linear, cfg=cfg, sta=sta)
    obj.main()
    vp_img_linear = obj.vp_img_arr

# color equalization
if cfg.params[cfg.opt_colo]:
    obj = pcam.lfp_extractor.LfpColorEqualizer(vp_img_arr=vp_img_linear, cfg=cfg, sta=sta)
    obj.main()
    vp_img_linear = obj.vp_img_arr

# copy light-field for refocusing process prior to contrast alignment and export
vp_img_arr = vp_img_linear.copy() if vp_img_linear is not None else None

# color management automation
obj = pcam.lfp_extractor.LfpContrast(vp_img_arr=vp_img_arr, cfg=cfg, sta=sta)
obj.main()
vp_img_arr = obj.vp_img_arr

# reduction of hexagonal sampling artifacts
if cfg.params[cfg.opt_arti]:
    obj = pcam.lfp_extractor.HexCorrector(vp_img_arr=vp_img_arr, cfg=cfg, sta=sta)
    obj.main()
    vp_img_arr = obj.vp_img_arr

# write viewpoint data to hard drive
if cfg.params[cfg.opt_view]:
    obj = pcam.lfp_extractor.LfpExporter(vp_img_arr=vp_img_arr, cfg=cfg, sta=sta)
    obj.write_viewpoint_data()

In [ ]:
view_obj = pcam.lfp_extractor.LfpViewpoints(vp_img_arr=vp_img_arr)
vp_arr = view_obj.reorder_vp_arr(pattern='circle', lf_radius=3)

plt.rcParams["animation.html"] = "jshtml"
from matplotlib import animation

fig, ax = plt.subplots()
l = ax.imshow(vp_arr[0])
animate = lambda i: l.set_data(vp_arr[i])
anim = animation.FuncAnimation(fig, animate, frames=len(vp_arr), interval=50)
plt.close() # get rid of initial figure

anim

## Computational change of focus

The light-field's well known synthetic focus capability is managed by the `LfpRefocuser` class with exemplary parameter setting $a=[0,2]$ in the `cfg.params` dictionary and key `cfg.ran_refo` controling the refocused range.

In [ ]:
# set refocus range $a$
cfg.params[cfg.ran_refo] = [0, 2]
# skip status messages
cfg.params[cfg.opt_prnt] = False

refocuser = pcam.lfp_refocuser.LfpRefocuser(vp_img_arr=vp_img_linear, cfg=cfg)
refocuser.main()
refo_stack = refocuser.refo_stack

plt.rcParams["animation.html"] = "jshtml"
from matplotlib import animation

fig, ax = plt.subplots()
l = ax.imshow(refo_stack[0])
animate = lambda i: l.set_data(refo_stack[i])
anim = animation.FuncAnimation(fig, animate, frames=len(refo_stack), interval=1000)
plt.close() # get rid of initial figure

anim